### **Project Two: An Improved Experimental Setup for Conway's Game of Life**

*   Extend the `Life` model developed in Lab 6
*   Develop a parser for Run-Length-Encoded GoL patterns
*   Apply regular expressions to a parsing problem
*   Experiment with interactions between persistent patterns

9-10 April, David Lu

#### **Part One: GoL**

Modules, classes, and functions for simulating and animating Conway's Game of Life, lifted straight from Lab 6.

In [1]:
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from scipy.signal import correlate2d

%matplotlib inline
# Configure matplotlib's animation library to work in the browser.
matplotlib.rc('animation', html='jshtml')

In [2]:
def plot_2d_array(array, axes=None, title='', cmap='Blues', **options):
    """
    Plot the 2D array as an image on the given axes  1's will be dark blue, 0's will be light blue.

    :param axes: the axes to plot on, or None to use the `plt.gca()` (current axes)
    :param options: keyword arguments passed directly to `plt.imshow()`
           see https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.imshow.html
    """
    axes = axes or plt.gca()  # If not axes are provided, draw on current axes
    axes.set_title(title)
    # Turn off axes labels and tick marks
    axes.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False ,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False,)
    # Defaults for displaying a "matrix" with hard-pixel boundaries and (0,0) at top-left
    options = {**dict(interpolation='nearest', origin='upper'), **options}
    axes.imshow(array, cmap=cmap, **options)

In [3]:
class Life2D:
    """ An basic 2D Cellular Automata that implementes Conway's Game of Life """
    kernel = np.array([[1, 1, 1],
                       [1, 10, 1],
                       [1, 1, 1]])

    next_state = np.zeros(19, dtype=np.uint8)
    next_state[[3, 12, 13]] = 1

    def __init__(self, n_rows, n_cols=None):
        """ Construct a n_rows x n_cols 2D CA """
        n_cols = n_cols or n_rows  # i.e., n_cols if n_cols is not None and n_cols != 0 else n_rows
        self.state = np.zeros(shape=(n_rows, n_cols), dtype=np.uint8)

    def step(self):
        """Executes one time step. """
        transitions = correlate2d(self.state, self.kernel, mode='same', boundary='wrap')
        self.state = self.next_state[transitions]

    def draw(self, axes=None, step=''):
        axes = axes or plt.gca()   # use pyplot's current axes if none are provided.
        plot_2d_array(self.state, axes, title=f"Conway's Game of Life {step}")

In [4]:
class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number
    """

    def __init__(self, model, frames=50, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            self.model.step()
        self.model.draw(self.ax, step=step)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()          # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)     # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()               # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass

#### **Part Two: Improving the Experimental Setup**

Makes it much easier for the user to place a known pattern into the starting state.

In [5]:
import re
import os
import requests

In [6]:
# Taking advantage of the common web address format shared by the RLE files
# hosted on conwaylife.com, looks up the file for the pattern name provided
# and copies its contents into a local file bearing the pattern name.

def download_rle_file (filename):

    address = f'https://conwaylife.com/patterns/{filename}.rle'
    response = requests.get(address)

    with open (f'{filename}.rle', 'w') as file:
        file.write(response.text)

# Decodes a run-length encoded (RLE) file to reproduce
# the encoded pattern as a two-dimensional array.

def decode_rle_file (filename):

    with open (f'{filename}.rle', 'r') as file:
        contents = file.readlines()

    dimensions_line = next(line for line in contents if line.startswith('x'))
    rle_lines = [line for line in contents if not re.match(r'^[#x\s]', line)]
    ncols, nrows = re.findall(r'[xy].*?(\d+)', dimensions_line)
    rle = ''.join(rle_lines).replace('\n', '')

    decoded = np.zeros((int(nrows), int(ncols)), dtype = np.uint8)

    # Parser Without Regex

    # i = 0
    # for row in rle[:-1].split('$'):

    #     extra_delineation = row.replace('b', 'b ').replace('o', 'o ')
    #     sequence_of_items = extra_delineation.split()

    #     cursor = 0

    #     for item in sequence_of_items:
    #         run_count = int(item[:-1] or 1)
    #         if item.endswith('o'):
    #             decoded[i][cursor:cursor+run_count] = np.ones(run_count)
    #         cursor += run_count

    #     i += 1 if not row[-1].isdigit() else int(row[-1])

    sequence_of_items = re.findall(r'(\d*)([bo$])', rle[:-1])

    i = 0; cursor = 0
    for item in sequence_of_items:
        run_count = int(item[0] or 1)

        match item[1]:
            case 'b':
                cursor += run_count
            case 'o':
                decoded[i][cursor:cursor+run_count] = np.ones(run_count)
                cursor += run_count
            case '$':
                i += run_count
                cursor = 0

    return decoded

In [27]:
class ConvenientLife2D(Life2D):

    def add_named_pattern (self, pattern, row = 3, col = 3):

        if not os.path.exists(f'{pattern}.rle'):
            download_rle_file(pattern)

        decoded_pattern = decode_rle_file(pattern)
        rows, cols = decoded_pattern.shape

        self.state[row:row+rows,col:col+cols] = decoded_pattern

#### **Part Three: A Small Experiment**

*Om nom nom nom -- Cookie Monster*



In [29]:
# A bite-sized experiment

Test_1 = ConvenientLife2D(11)
Test_1.add_named_pattern('glider', row = 1, col = 1)
Test_1.add_named_pattern('eater1', row = 6, col = 6)

anim = Animation2D(Test_1, frames = 11, figsize = (5,5))
anim.show()

#### **Part Four: Complexity Analysis**

I'd reckon that it's probably on the order of $O(nm)$, but it seems I've run out of time to verify that now.

I really meant to finish this project on time, but instead I fell soundly asleep at 7pm, sorry.